# Pythonプログラミング入門 第6回
クラスの継承について説明します

# クラスの継承、基底・派生クラス

6-1 ではユーザが新しくクラスを定義する方法について説明しました。
クラスを定義する際にすべてを新しく定義するのではなく、すでに存在する類似したクラスを拡張できれば便利です。
そのような拡張はクラスの**継承**によって実現できます。ここで継承されるクラスを基底クラス（あるいは親クラス）、
継承したものを派生クラス（あるいは子クラス）と呼びます。  

実際、Python では多様な処理をおこなうための強力なソフトウェアフレームワークが用意されています。
フレームワークとは、お手本を集めたようなもので、
ユーザはお手本と異なるところを手直ししたり追加したりする、すなわちカスタマイズする、ことによって、
フレームワークを活用します。
このカスタマイズにあたって、ユーザはフレームワーク側で用意された基底クラスを拡張した派生クラスをつくっていく方法が一般的です。

## 派生クラスの定義

基底クラス BaseClass を継承し、派生クラス DerivedClass を定義するには以下のようにおこないます。

---
```Python
class DerivedClass(BaseClass):
    派生クラスに関するプログラム文
```

---

派生クラスでは、それの基底クラスの属性・メソッドをそのまま利用できるので、
派生クラスに依存した属性・メソッドのみをプログラムすれば良いことになります。

以下の例では、2 次元平面上の点を表すオブジェクトのクラス `Point2DInitialized`を基底クラスとして、
それを継承する派生クラス `Point2DPolar` を定義しています。  
`Point2DPolar` ではメソッド `__init__()`, ` dist_from_origin()` や、属性 `x, y` は
全く定義していませんが、基底クラスのものをそのまま使うことができます。  
もちろん新しく定義したメソッド `r()`, `theta()` も使えます。

In [1]:
import math                                 # 逆 cos 関数に必要なため

class Point2DInitialized:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def dist_from_origin(self):
        return (self.x**2 + self.y**2)**0.5
    
class Point2DPolar(Point2DInitialized):
    def r(self):
        return(self.dist_from_origin())
    def theta(self):
        if self.dist_from_origin != 0:
            return math.acos( self.x / self.dist_from_origin())
        else:
            return 0

In [2]:
p2p = Point2DPolar(3, 4)
p2p

In [3]:
p2p.x, p2p.y           # Point2DPolar では x, y 属性は定義していないが、基底クラスの属性が使える

(3, 4)

In [4]:
p2p.dist_from_origin() # メソッドも基底クラスのものが使える

5.0

In [5]:
p2p.r()                # 新しく定義したメソッド r()

5.0

In [6]:
p2p.theta()

0.9272952180016123

## 属性・メソッドの上書き

派生クラスで基底クラスの属性・メソッドを上書き（オーバーライド）することもできます。
以下の例では `dist_from_origin` を演算子 `**` を利用したものから、
数学関数モジュールの平方根を求める関数 `math.sqrt()`に置き換えています。

In [7]:
import math

class Point2DPolar2(Point2DPolar):
    def dist_from_origin(self):
        print("new dist_from_origin")
        return math.sqrt(self.x**2 + self.y**2)

In [8]:
p2p2 = Point2DPolar2(3, 4)
p2p2.dist_from_origin()

new dist_from_origin


5.0

## 基底クラスのメソッド呼び出し

派生クラスのメソッドから基底クラスのメソッド・属性を呼びだしたいことはよくあります。
このような場合組み込み関数 `super()`を利用します。

以下は `Point2DPolar` を継承したクラス、`Point2DCached` の例です。
`Point2DCached` では `x, y` が与えられる初期化時に極座標も先に計算します。

派生クラス `Point2DCached`では初期化メソッド `__init__()`　を置き換えており、
その中で基底クラスの初期化メソッドを `super().__init__()` を使って呼び出しています。

In [9]:
class Point2DCached(Point2DPolar):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.r_c = (x*x + y*y)**0.5
        self.theta_c = self.theta()
    def dist_from_origin(self):
        return self.r_c

In [10]:
p2c = Point2DCached(3, 4)
p2c.x, p2c.y

(3, 4)

In [11]:
p2c.r_c, p2c.theta_c

(5.0, 0.9272952180016123)

## クラスとイテレータ

これまで説明したように `for` 文を用いることで、プログラムはリスト・辞書といったデータ構造やファイルにわたって反復的にアクセスすることができます。

---
```Python
for i in [1, 2, 3, 4]:
    print(i)

for line in open("myfile,txt"):
   print(line)

```

`for` 文では、第4回に説明した組み込み関数 `iter()` を呼び、イテラブルオブジェクトからイテレータを作成します。
その後 `next()` を繰り返し呼び、要素を一つずつ取り出しています。

ここでは、このような反復処理をサポートするクラスの定義方法を説明します。
これには、２つのメソッド`__iter__()` と `__next__()` を定義します。

前者の `__iter__()`はオブジェクトに対応するイテレータを返します。
クラスで `__next__()` が定義されていればオブジェクトそのもの、すなわち `self`を返します。

後者の `__next__()` は、呼ばれるたびにデータを一つずつ返します。返すデータがなくなれば、`raise StopIteration`
で例外（エラー）を送出します。
例外を受けプログラムは `for`ループの繰り返しから脱けます。

具体例として、以下の折れ線を扱うクラス`Chain`を考えます。
`Chain` は座標 $(p_1, p_2, ...)$ を要素とする `series` 属性をもちます。

In [12]:
class Chain():
    def __init__(self, series):
        self.series = series
        self.index = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.index == len(self.series):
            raise StopIteration
        self.index += 1
        return self.series[self.index - 1]

ここで、オブジェクトを生成し `for` ループでアクセスしてみます。

In [13]:
ch = Chain([(0,0), (0,1), (1,1), (1,0)])
for p in ch:
    print(p)

(0, 0)
(0, 1)
(1, 1)
(1, 0)


ただし、一度繰り返しが終了すると、属性 `Chain.index` が最大値となっているためもう繰り返しはおこないません。

In [15]:
for p in ch:
    print(p)

## 継承関係を検査する関数

`issubclass(clas1, class2)` は `class1` が `class2` の継承関係を検査する組み込み関数です:

In [16]:
issubclass(Point2DCached, Point2DPolar)       # Point2DCached は　Point2DPolar を継承している

True

In [17]:
issubclass(Point2DPolar, Point2DCached)       # 逆は正しくない

False

In [18]:
issubclass(Point2DCached, Point2DInitialized) # 継承関係が入れ子になっていても検査できます

True

## 予習課題
以下のセルに講義情報をあつかうクラス `SimpleCourse` クラスを定義しています。
このクラスでは、日本語タイトル・日本語教員名・開講時間を属性として持ちます。

これの派生クラスとして、以下の条件を満たす `SimpleCourseE`を作成してください:

1. 英語タイトル(title)・英語教員名(name)属性もオブジェクト生成時に初期化する。
    オブジェクト生成時の引数の与え方は以下を参考にしてください。
1. 2 つのメソッド、英語タイトル・英語教員名を文字列として返す `info_e()` と、日本語タイトル・日本語教員名を返す `info_j()` を実装してください。
    返り値の形式は `(タイトル、教員名)`の tuple としてください。

---
```Python

sce = SimpleCourseE("コンピュータシステム概論", "Overview of Computer Systems", "小林克志", "Katsushi Kobayashi","木5")

```
---

In [53]:
class SimpleCourse:
    def __init__(self, title_j, name_j, slot):
        self.title_j = title_j
        self.name_j = name_j
        self.slot = slot

sc = SimpleCourse("コンピュータシステム概論", "小林克志", "木5")

class SimpleCourseE(SimpleCourse):
    def __init__(self, title_j, title_e, name_j, name_e, slot):
        super().__init__(title_j, name_j, slot)
        self.title_e = title_e
        self.name_e = name_e
    def info_e(self):
        return (self.title_e, self.name_e)
    def info_j(self):
        return (self.title_j, self.name_j)
    
sce = SimpleCourseE("コンピュータシステム概論", "Overview of Computer Systems", "小林克志", "Katsushi Kobayashi","木5")

In [54]:
sce.info_j()

('コンピュータシステム概論', '小林克志')

In [55]:
sce.info_e()

('Overview of Computer Systems', 'Katsushi Kobayashi')